In [1]:
import glob
import spacy
import re
import pandas as pd

In [2]:
df = pd.read_csv("data/filenames.csv")
df

,Archive ID,Title,Volume,Image Count,SI Libraries URL,BHL URL,Internet Archive URL
0,annualreportofbo1862smit,Annual report of the Board of Regents of the S...,1862,464,https://library.si.edu/digital-library/book/an...,http://www.biodiversitylibrary.org/ia/annualre...,https://archive.org/details/annualreportofbo18...
1,annualreportofbo1908smitfo,Annual report of the Board of Regents of the S...,1908,972,https://library.si.edu/digital-library/book/an...,http://www.biodiversitylibrary.org/ia/annualre...,https://archive.org/details/annualreportofbo19...
2,smithsonianyear1974smit,Smithsonian year,1974,526,https://library.si.edu/digital-library/book/sm...,http://www.biodiversitylibrary.org/ia/smithson...,https://archive.org/details/smithsonianyear197...
3,annualreportofbo1956smit,Annual report of the Board of Regents of the S...,1956,710,https://library.si.edu/digital-library/book/an...,http://www.biodiversitylibrary.org/ia/annualre...,https://archive.org/details/annualreportofbo19...
4,annualreportofbo1918smit,Annual report of the Board of Regents of the S...,1918,724,https://library.si.edu/digital-library/book/an...,http://www.biodiversitylibrary.org/ia/annualre...,https://archive.org/details/annualreportofbo19...
...,...,...,...,...,...,...,...
193,annualreportofbo1943smit,Annual report of the Board of Regents of the S...,1943,792,https://library.si.edu/digital-library/book/an...,http://www.biodiversitylibrary.org/ia/annualre...,https://archive.org/details/annualreportofbo19...
194,annualreportofbo1877smit,Annual report of the Board of Regents of the S...,1877,516,https://library.si.edu/digital-library/book/an...,http://www.biodiversitylibrary.org/ia/annualre...,https://archive.org/details/annualreportofbo18...
195,annualreportofbo1876smith,Annual report of the Board of Regents of the S...,1876,544,https://library.si.edu/digital-library/book/an...,http://www.biodiversitylibrary.org/ia/annualre...,https://archive.org/details/annualreportofbo18...
196,annualreportfory1951united,Annual report for the year ended June 30 ...,1951,134,https://library.si.edu/digital-library/book/an...,http://www.biodiversitylibrary.org/ia/annualre...,https://archive.org/details/annualreportfory19...


In [4]:
nlp = spacy.blank("en")
nlp.add_pipe("sentencizer")

In [5]:
files = glob.glob("data/annual_reports/*.txt")
len(files)

198

In [6]:
archive_ids = df["Archive ID"].tolist()

In [50]:
all_data = []
index = []
x=0
for file in files:
    archive_id = file.split("\\")[-1].replace(".txt", "").split("_")[0]
    year = str(df.loc[df["Archive ID"] == archive_id].Volume.tolist()[0]).split()[0]
    title = df.loc[df["Archive ID"] == archive_id].Title.tolist()[0]

    with open(file, "r", encoding="utf-8") as f:
        data = f.read()
    data = re.sub("\[([^\)]+)\]", "", data)
    while "  " in data:
        data = data.replace("  ", " ")
    while "\n\n\n" in data:
        data = data.replace("\n\n\n", "\n\n")
    # sentences = list(nlp(data).sents)
    paras = data.split("\n\n")
    # for sent in sentences:
    #     if len(sent) > 8:
    for para in paras:
        para = para.replace("\n- ", "")
        para = para.replace("\n-", "")
        para = para.replace("\n", " ")
        para = para.replace("- ", "")
        if len(para.split()) > 20:
            words = 0
            for word in para.split():
                if any(char.isdigit() for char in word):
                    pass
                else:
                    words = words+1
            if words > 20:
                all_data.append((x, para, year))
        index.append((archive_id, year, para))
        x=x+1
len(all_data)

161592

In [54]:
print(all_data[6000:6001])

[(26039, 'The U.S. Mint was founded in 1791, and the national numismatic collection was started soon after, at Philadelphia, site of the first mint.', '1961')]


In [31]:
index[40]

('annualreportfory1951united',
 '1951',
 'In the biological sciences research provides the data essential to a further study of the relationships of the various forms of life to each other and to their environment, their evolutionary trends, their fundamental properties, and their economic significance.')

In [32]:
# Get started in a couple lines
from txtai.embeddings import Embeddings

In [55]:
embeddings = Embeddings({"path": "sentence-transformers/all-MiniLM-L6-v2", "content": True})

In [ ]:
embeddings.index(all_data)

In [ ]:
res = embeddings.search("woman in science", 1)
for i, item in enumerate(res):
    print(item["id"])
    print(f"{i+1:02}:", item["text"])
    print()
    print(index[int(item["id"])])
    # print(f"Context: ", index[int(item["id"])-1:int(item["id"])+2])

In [ ]:
embeddings.save("models/annual_reports_paras")